In [10]:
import pandas as pd 
df = pd.read_csv('train.csv')
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [14]:
print(df.shape)
#  Excluindo colunas com valores ausentes
df.dropna(axis=1, inplace=True)
print(df.shape)


(1460, 81)
(1460, 62)


Define Função para cálculo rápido do melhor score de preprocessamento. <br>
Será Testado entre: <br>
#1 - Retirar os dados não numéricos. <br>
#2 - Codificação Ordinal. <br>
#3 - Codificação One Hot <br>

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
 
def model_score(X, y):
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
    model = RandomForestRegressor(random_state=1)
    model.fit(train_X, train_y)
    preds = model.predict(val_X)
    return mean_absolute_error(val_y, preds)

#1 - Retirar os dados não numéricos.

In [15]:
# Define X and y
y = df.SalePrice
X = df.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

model_score(X, y)

17006.18194520548

#2 - Codificação Ordinal.

In [19]:
from sklearn.preprocessing import OrdinalEncoder

object_cols = [col for col in df.columns if df[col].dtype == "object"]

y = df.SalePrice
X = df.drop(['SalePrice'], axis=1)
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(df[object_cols])
model_score(X, y)

16437.31802739726

#3 - Codificação One Hot <br>

In [28]:
from sklearn.preprocessing import OneHotEncoder

object_cols = [col for col in df.columns if df[col].dtype == "object"]
y = df.SalePrice
X = df.drop(['SalePrice'], axis=1)
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(df[object_cols]))
OH_cols.index = X.index
num_X = df.drop(object_cols, axis=1).drop(['SalePrice'], axis=1)
OH_X = pd.concat([num_X, OH_cols], axis=1)
OH_X.columns = [str(i) for i in OH_X.columns]
model_score(OH_X, y)

16586.494547945204

Foi decidido utilizar o método de #2 - Codificação Ordinal. por ter um MSE mais baixo. <br>
Com isso foi criando uma função para pré processamento dos dados:

In [29]:
def preprocess(df):
    object_cols = [col for col in df.columns if df[col].dtype == "object"]

    y = df.SalePrice
    X = df.drop(['SalePrice'], axis=1)
    ordinal_encoder = OrdinalEncoder()
    X[object_cols] = ordinal_encoder.fit_transform(df[object_cols])
    return X, y

Testar modelo de arvore de decisão

In [31]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X, y = preprocess(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
dt = DecisionTreeRegressor(random_state=0)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print('MSE: ', mean_absolute_error(y_test, y_pred))


MSE:  25057.701369863014


Será Utilizado o a Biblioteca Pycarte para comparar os modelos de aprendizado de máquina #pip install pycaret (reqs: python<3.8) <br>

In [33]:
from pycaret.regression import *
X, y = preprocess(df)
X['SalePrice'] = y
exp_reg = setup(data = X, target = 'SalePrice', session_id=123, normalize = True, transformation = True)


,Description,Value
0,Session id,123
1,Target,SalePrice
2,Target type,Regression
3,Original data shape,"(1460, 62)"
4,Transformed data shape,"(1460, 62)"
5,Transformed train set shape,"(1021, 62)"
6,Transformed test set shape,"(439, 62)"
7,Numeric features,61
8,Preprocess,True
9,Imputation type,simple


In [34]:
exp_reg.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,17062.4064,789040304.7019,27411.3775,0.8789,0.1364,0.0983,0.7780
lightgbm,Light Gradient Boosting Machine,17573.5682,982714901.3682,30318.2912,0.8539,0.1448,0.1023,0.7200
rf,Random Forest Regressor,18558.7684,1035873814.5819,31288.5652,0.8447,0.1525,0.1097,0.9380
et,Extra Trees Regressor,18501.1425,1170340393.0537,32818.1099,0.8309,0.1498,0.1078,0.8840
ada,AdaBoost Regressor,24708.2228,1244401593.5016,34893.4518,0.8112,0.2003,0.1604,0.7960
huber,Huber Regressor,21935.6745,1493659352.3201,37319.8228,0.7799,0.1919,0.1240,0.6530
par,Passive Aggressive Regressor,22206.0027,1521067262.6484,37742.7655,0.7764,0.1781,0.1225,0.6700
en,Elastic Net,22791.1364,1523389651.8956,37795.3737,0.7753,0.1821,0.1291,0.6430
br,Bayesian Ridge,23443.2026,1622955609.3098,38697.3478,0.7574,0.1967,0.1386,0.6310
knn,K Neighbors Regressor,24205.3973,1666248983.2854,39862.2397,0.7536,0.1867,0.1390,0.7250


GradientBoostingRegressor(random_state=123)

Definindo o modelo Gradient Boosting Regressor	como uma variavel do SKlearn

In [35]:
gbr = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,19893.0338,1725055138.3597,41533.7831,0.7893,0.1727,0.1098
1,17271.7438,907318459.9355,30121.7274,0.8584,0.1286,0.0884
2,15055.9986,453183129.0454,21288.0983,0.9019,0.1261,0.0943
3,15655.7549,607597567.6317,24649.4943,0.8829,0.1256,0.0901
4,18511.6683,886099129.0064,29767.4172,0.9099,0.1711,0.1219
5,14823.6981,444000729.3459,21071.3248,0.9385,0.1235,0.0900
6,19048.4575,807780678.4943,28421.4827,0.8287,0.1290,0.1013
7,16190.1123,541688136.1101,23274.1946,0.9205,0.1283,0.0955
8,19596.7702,1055573848.5504,32489.5960,0.8425,0.1500,0.1105


AttributeError: 'GradientBoostingRegressor' object has no attribute 'analyze_model'

In [36]:
evaluate_model(gbr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [37]:
tuned_gbr = tune_model(gbr)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,23286.3062,2511127284.4892,50111.1493,0.6933,0.2086,0.1384
1,17462.9421,700409440.6349,26465.2497,0.8907,0.1345,0.0947
2,16325.2191,575228848.7229,23983.9290,0.8755,0.1393,0.1025
3,17389.1846,637685273.3720,25252.4310,0.8771,0.1285,0.0991
4,19898.4140,1317908646.8867,36303.0115,0.8659,0.1601,0.1158
5,16560.3621,592445084.3583,24340.1948,0.9179,0.1498,0.1038
6,20806.7207,1025514371.6535,32023.6533,0.7825,0.1381,0.1087
7,17052.2353,630157489.6835,25102.9379,0.9075,0.1290,0.0955
8,19290.7715,1495787678.9687,38675.4144,0.7768,0.1641,0.1135


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
